In [ ]:
! pip install datasets

In [ ]:
! pip install transformers[torch]

In [4]:
from transformers import BertTokenizer, BertForSequenceClassification, BertModel,Trainer, TrainingArguments
from datasets import load_dataset, load_metric
import torch
import numpy as np

# BERT 톺아보기

In [5]:
# model 불러오기

model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
# max_position_embedding : 512 (한 번에 input으로 줄 수 있는 최대 토큰 수)
# vocab_size : 30522 개의 단어

print(model.config)

BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.41.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [7]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

### tokenizer

In [8]:
# tokenizer 불러오기

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [9]:
## tokenizer 사용 예시

n = 10  # input의 token의 최대 개수

text = 'I love Large Language Model.'

tokens = tokenizer.tokenize(text)
print('tokenization 후:',tokens)

tokens = ['[CLS]'] + tokens + ['[SEP]']
tokens += ['[PAD]'] * (n - len(tokens))

print('special token 추가:',tokens)

attention_mask = [0 if token=='[PAD]' else 1 for token in tokens]
print('attention mask:',attention_mask)

token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('token_ids:', token_ids)
print()


token_ids = torch.tensor(token_ids).reshape(1,-1)
attention_mask = torch.tensor(attention_mask).reshape(1,-1)

print('token_ids shape:', token_ids.shape)
print('attention_mask:', attention_mask.shape)

tokenization 후: ['i', 'love', 'large', 'language', 'model', '.']
special token 추가: ['[CLS]', 'i', 'love', 'large', 'language', 'model', '.', '[SEP]', '[PAD]', '[PAD]']
attention mask: [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
token_ids: [101, 1045, 2293, 2312, 2653, 2944, 1012, 102, 0, 0]

token_ids shape: torch.Size([1, 10])
attention_mask: torch.Size([1, 10])


In [10]:
# model의 output 관찰

output = model(token_ids, attention_mask = attention_mask)

In [11]:
output.last_hidden_state.shape

torch.Size([1, 10, 768])

In [12]:
output.pooler_output.shape

torch.Size([1, 768])

In [13]:
# 맨 앞의 CLS token의 last_hidden_state와 pooler_output이 같은 tensor인가? -> False

torch.equal(output.last_hidden_state[0][0], output.pooler_output[0])

False

In [14]:
# 직접 pooling 통과시키기 -> same
# output.pooler_output == hidden_size -> hidden_size linear 통과 후 tanh를 씌운 값

pooler_linear = torch.nn.Linear(model.config.hidden_size, model.config.hidden_size)
pooler_activation = torch.nn.Tanh()

pooler_linear.weight.data = model.pooler.dense.weight.data
pooler_linear.bias.data = model.pooler.dense.bias.data

pooler_output_direct = pooler_activation(pooler_linear(output.last_hidden_state[0][0]))

torch.allclose(pooler_output_direct, output.pooler_output)

True

In [15]:
# model 선언 때 output_hidden_states = True 로 설정한다면 각 Layer에서 얻은 vector 반환 가능

model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)

output = model(token_ids, attention_mask = attention_mask)

print('hidden_states의 개수:',len(output.hidden_states))

# 마지막 Layer를 통과한 결과와 last_hidden_state의 결과는 같음.
print(torch.equal(output.hidden_states[-1][0], output.last_hidden_state[0]))

hidden_states의 개수: 13
True


# BERT Fine-tuning with IMDb dataset

### transformers library의 trainer 활용하여 모델 전체 학습

In [16]:
dataset = load_dataset('imdb')

print(type(dataset))
print(dataset)
print()
print(dataset['train'])

<class 'datasets.dataset_dict.DatasetDict'>
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [17]:
# data example
# I Am Curious(Yellow)에 대한 감상평으로 보임.
# 긍정인 경우 1, 부정인 경우 0

print('text data example : ' , dataset['train']['text'][0])
print()
print('label example : ', dataset['train']['label'][0])

text data example :  I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few

In [18]:
# dataset 준비

temp = dataset['train'].train_test_split(test_size = 0.5)
train_data, test_data = temp['train'], temp['test']

temp = train_data.train_test_split(test_size = 0.25)
train_data, val_data = temp['train'], temp['test']

def preprocess(data):
  return tokenizer(data['text'], padding='max_length', truncation=True)

encoded_train = train_data.map(preprocess, batched=True, batch_size=64)
encoded_val = val_data.map(preprocess, batched=True, batch_size=64)
encoded_test = test_data.map(preprocess, batched=True, batch_size=64)

Map:   0%|          | 0/9375 [00:00<?, ? examples/s]

Map:   0%|          | 0/3125 [00:00<?, ? examples/s]

Map:   0%|          | 0/12500 [00:00<?, ? examples/s]

In [19]:
encoded_train.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
encoded_val.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
encoded_test.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [20]:
# training args

batch_size = 32

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size= batch_size,
    per_device_eval_batch_size= batch_size*2,
    num_train_epochs=3,
    weight_decay=0.01,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
model_classifier = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
trainer = Trainer(
    model = model_classifier,
    args = training_args,
    train_dataset = encoded_train,
    eval_dataset = encoded_val
)

In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.232349
2,0.238100,0.245271
3,0.238100,0.292515


TrainOutput(global_step=879, training_loss=0.1744942518631346, metrics={'train_runtime': 2910.8543, 'train_samples_per_second': 9.662, 'train_steps_per_second': 0.302, 'total_flos': 7399998432000000.0, 'train_loss': 0.1744942518631346, 'epoch': 3.0})

### Only last linear layer만 fine-tuning

In [20]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data1, data2, labels):
        self.data1 = data1
        self.data2 = data2
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        token_ids = self.data1[idx]
        attention_mask = self.data2[idx]
        label = self.labels[idx]
        return token_ids, attention_mask, label

dataset = CustomDataset(encoded_train['input_ids'], encoded_train['attention_mask'], encoded_train['label'])

batch_size = 32
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [21]:
import torch.nn as nn
import torch.optim as optim


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BERT = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)

class ImdbClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.BERT = BERT.to(device)
    self.linear = nn.Linear(768,2)

  def forward(self, input_ids, attention_mask):
    x = self.BERT(input_ids , attention_mask = attention_mask)
    x = self.linear(x.last_hidden_state[:,0,:])

    return x

model = ImdbClassifier()
model.to(device)

ImdbClassifier(
  (BERT): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.linear.parameters(), lr=0.001)

epochs = 2

torch.cuda.empty_cache()

for epoch in range(epochs):
  model.train()
  for input_ids, attention_mask, label in train_loader:

    input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)

    optimizer.zero_grad()

    output = model(input_ids, attention_mask)

    loss = loss_function(output, label)
    loss.backward()
    optimizer.step()